In [1]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import os

target_date = "2015-07" #예측 날짜
seed_word_dict = {"hawkish": ['높', '팽창', '인상', '매파', '성장', '투기_억제', '상승',  '인플레이션_압력', '증가', '위험_선호', '상회', '물가_상승', '과열', '금리_상승', '확장', '상방_압력', '긴축', '변동성_감소', '흑자', '채권_가격_하락', '견조', '요금_인상', '낙관', '부동산_가격_상승', '상향']
                
        
                ,"doveish":["낯", "축", "인하", "비둘기", "둔화", "악화", "하락", "회복_못하", "감소", "위험_회피", "하회", "물가_하락", "위축", "금리_하락", "침체", "하방_압력", "완화", "변동성_확대", "적자", "채권_가격_상승", "부진", "요금_인하", "비관", "부동산_가격_하락", "하향"]}


train_df = pd.read_csv("./train_df.csv")


In [8]:
train_df

,words,label,1_ngrams,2_ngrams,3_ngrams,4_ngrams,5_ngrams,new_column
0,월 기점 채권 금리 하락세 변곡점 형성 전망 요약 결론 추가 금리 인하 불확실성 경...,doveish,월 기점 채권 금리 하락세 변곡점 형성 전망 요약 결론 추가 금리 인하 불확실성 경...,월_기점 기점_채권 채권_금리 금리_하락세 하락세_변곡점 변곡점_형성 형성_전망 전...,월_기점_채권 기점_채권_금리 채권_금리_하락세 금리_하락세_변곡점 하락세_변곡점_...,월_기점_채권_금리 기점_채권_금리_하락세 채권_금리_하락세_변곡점 금리_하락세_변...,월_기점_채권_금리_하락세 기점_채권_금리_하락세_변곡점 채권_금리_하락세_변곡점_...,월 기점 채권 금리 하락세 변곡점 형성 전망 요약 결론 추가 금리 인하 불확실성 경...
1,일 금리 인하 요약 월기 금리 인하 이후 도금 통 위 기대감 지속 월금 통 위 글로...,doveish,일 금리 인하 요약 월기 금리 인하 이후 도금 통 위 기대감 지속 월금 통 위 글로...,일_금리 금리_인하 인하_요약 요약_월기 월기_금리 금리_인하 인하_이후 이후_도금...,일_금리_인하 금리_인하_요약 인하_요약_월기 요약_월기_금리 월기_금리_인하 금리...,일_금리_인하_요약 금리_인하_요약_월기 인하_요약_월기_금리 요약_월기_금리_인하...,일_금리_인하_요약_월기 금리_인하_요약_월기_금리 인하_요약_월기_금리_인하 요약...,일 금리 인하 요약 월기 금리 인하 이후 도금 통 위 기대감 지속 월금 통 위 글로...
2,월금 통 위 이후 향후 금리 방향 점검 요약 지난주 기준금리 인하 결정 채권시장 약...,doveish,월금 통 위 이후 향후 금리 방향 점검 요약 지난주 기준금리 인하 결정 채권시장 약...,월금_통 통_위 위_이후 이후_향후 향후_금리 금리_방향 방향_점검 점검_요약 요약...,월금_통_위 통_위_이후 위_이후_향후 이후_향후_금리 향후_금리_방향 금리_방향_...,월금_통_위_이후 통_위_이후_향후 위_이후_향후_금리 이후_향후_금리_방향 향후_...,월금_통_위_이후_향후 통_위_이후_향후_금리 위_이후_향후_금리_방향 이후_향후_...,월금 통 위 이후 향후 금리 방향 점검 요약 지난주 기준금리 인하 결정 채권시장 약...
3,강세 준비 숨고르기 요약 강세 준비 숨고르기 금주 채권 금리 월금 통 위 이후 추가...,doveish,강세 준비 숨고르기 요약 강세 준비 숨고르기 금주 채권 금리 월금 통 위 이후 추가...,강세_준비 준비_숨고르기 숨고르기_요약 요약_강세 강세_준비 준비_숨고르기 숨고르기...,강세_준비_숨고르기 준비_숨고르기_요약 숨고르기_요약_강세 요약_강세_준비 강세_준...,강세_준비_숨고르기_요약 준비_숨고르기_요약_강세 숨고르기_요약_강세_준비 요약_강...,강세_준비_숨고르기_요약_강세 준비_숨고르기_요약_강세_준비 숨고르기_요약_강세_준...,강세 준비 숨고르기 요약 강세 준비 숨고르기 금주 채권 금리 월금 통 위 이후 추가...
4,미국 금리 급등 스캔들 요약 이번 주채 시장 전망 약화 정책 기대 미국 금리 급등 ...,doveish,미국 금리 급등 스캔들 요약 이번 주채 시장 전망 약화 정책 기대 미국 금리 급등 ...,미국_금리 금리_급등 급등_스캔들 스캔들_요약 요약_이번 이번_주채 주채_시장 시장...,미국_금리_급등 금리_급등_스캔들 급등_스캔들_요약 스캔들_요약_이번 요약_이번_주...,미국_금리_급등_스캔들 금리_급등_스캔들_요약 급등_스캔들_요약_이번 스캔들_요약_...,미국_금리_급등_스캔들_요약 금리_급등_스캔들_요약_이번 급등_스캔들_요약_이번_주...,미국 금리 급등 스캔들 요약 이번 주채 시장 전망 약화 정책 기대 미국 금리 급등 ...
...,...,...,...,...,...,...,...,...
154441,표 행간 단기 금리 기준 코리보 일 다음 일 전시 고시 한국 은행 단기 금리 기준 ...,hawkish,표 행간 단기 금리 기준 코리보 일 다음 일 전시 고시 한국 은행 단기 금리 기준 ...,표_행간 행간_단기 단기_금리 금리_기준 기준_코리보 코리보_일 일_다음 다음_일 ...,표_행간_단기 행간_단기_금리 단기_금리_기준 금리_기준_코리보 기준_코리보_일 코...,표_행간_단기_금리 행간_단기_금리_기준 단기_금리_기준_코리보 금리_기준_코리보_...,표_행간_단기_금리_기준 행간_단기_금리_기준_코리보 단기_금리_기준_코리보_일 금...,표 행간 단기 금리 기준 코리보 일 다음 일 전시 고시 한국 은행 단기 금리 기준 ...
154442,채권 오전 금리 상승국 고년 입찰 여파 기자 국고채 금리 오전 중 상승 채권시장 따...,hawkish,채권 오전 금리 상승국 고년 입찰 여파 기자 국고채 금리 오전 중 상승 채권시장 따...,채권_오전 오전_금리 금리_상승국 상승국_고년 고년_입찰 입찰_여파 여파_기자 기자...,채권_오전_금리 오전_금리_상승국 금리_상승국_고년 상승국_고년_입찰 고년_입찰_여...,채권_오전_금리_상승국 오전_금리_상승국_고년 금리_상승국_고년_입찰 상승국_고년_...,채권_오전_금리_상승국_고년 오전_금리_상승국_고년_입찰 금리_상승국_고년_입찰_여...,채권 오전 금리 상승국 고년 입찰 여파 기자 국고채 금리 오전 중 상승 채권시장 따...
154443,국채선물 낙폭 확대 입찰 소화 기자 국채선물 오후 낙폭 확대 채권시장 따르 국채선물...,hawkish,국채선물 낙폭 확대 입찰 소화 기자 국채선물 오후 낙폭 확대 채권시장 따르 국채선물...,국채선물_낙폭 낙폭_확대 확대_입찰 입찰_소화 소화_기자 기자_국채선물 국채선물_오...,국채선물_낙폭_확대 낙폭_확대_입찰 확대_입찰_소화 입찰_소화_기자 소화_기자_국채...,국채선물_낙폭_확대_입찰 낙폭_확대_입찰_소화 확대_입찰_소화_기자 입찰_소화_기자...,국채선물_낙폭_확대_입찰_소화 낙폭_확대_입찰_소화_기자 확대_입찰_소화_기자_국채...,국채선물 낙폭 확대 입찰 소화 기자 국채선물 오후 낙폭 확대 채권시장 따르 국채선물...
154444,금리 상승 국채선물 하락 은행권 비드 기자 금리스와프 금리 상승 마감 국채선물 하락...,hawkish,금리 상승 국채선물 하락 은행권 비드 기자 금리스와프 금리 상승 마감 국채선물 하락...,금리_상승 상승_국채선물 국채선물_하락 하락_은행권 은행권_비드 비드_기자 기자_금...,금리_상승_국채선물 상승_국채선물_하락 국채선물_하락_은행권 하락_은행권_비드 은행...,금리_상승_국채선물_하락 상승_국채선물_하락_은행권 국채선물_하락_은행권_비드 하락...,금리_상승_국채선물_하락_은행권 상승_국채선물_하락_은행권_비드 국채선물_하락_은행...,금리 상승 국채선물 하락 은행권 비드 기자 금리스와프 금리 상승 마감 국채선물 하락...


In [9]:
train_list = train_df['words'].tolist()

In [13]:
X_train = train_list

# 2. 특성 추출 (형태소 분석 및 N-그램)
vocab = [token.split() for token in X_train]

# 3. N-그램 추출
phrases = Phrases([x for x in vocab], min_count=1, threshold=1.1)
ngram_X_train = [' '.join(phrases[x]) for x in vocab]
New_test_ngrams = ngram_X_train

# 4. Ngram2Vec 모델 학습
model = Word2Vec([x.split() for x in New_test_ngrams], vector_size=300, window=5, min_count=25)

In [11]:
# 이걸로 테스트

# 넘파이 코사인 유사도 계산
def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

def ngram2vec_train(train_list):
    X_train = train_list

    # 2. 특성 추출 (형태소 분석 및 N-그램)
    vocab = [token.split() for token in X_train]

    # 3. N-그램 추출
    phrases = Phrases([x for x in vocab], min_count=1, threshold=1.1)
    ngram_X_train = [' '.join(phrases[x]) for x in vocab]
    New_test_ngrams = ngram_X_train

    # 4. Ngram2Vec 모델 학습
    model = Word2Vec([x.split() for x in New_test_ngrams], vector_size=300, window=5, min_count=25)
    return model,phrases
model,phrases = ngram2vec_train(train_list)
print("모델 생성 하였습니다")

# 극성 기준 사전
doveish_word_list = []
hawkish_word_list = []
for key in seed_word_dict['doveish']:
    try:
        target = model.wv[key]
        doveish_word_list.append(key)
    except:
        pass

for key in seed_word_dict['hawkish']:
    try:
        target = model.wv[key]
        hawkish_word_list.append(key)
    except:
        pass
model_keys = model.wv.index_to_key
print("극성 기준 사전 생성 하였습니다")

# 카운팅할 극성 워드 생성
Word_Emotion_dict = {"doveish":[],"hawkish":[]}
for key in model_keys:
    doveish_score = 0
    hawkish_score = 0
    target = model.wv[key]
    for word in doveish_word_list:
        doveish_target = model.wv[word]
        doveish_score += cos_sim(target, doveish_target)
    for word in hawkish_word_list:
        hawkish_target = model.wv[word]
        hawkish_score += cos_sim(target, hawkish_target)
    if doveish_score / len(doveish_word_list) > hawkish_score / len(hawkish_word_list):
        #print(f"{key} 해당 키는 비둘기적입니다")
        Word_Emotion_dict["doveish"].append(key)
    elif doveish_score / len(doveish_word_list) == hawkish_score / len(hawkish_word_list):
        print(f"{key} 해당 키는 중립적입니다")
    elif doveish_score / len(doveish_word_list) < hawkish_score / len(hawkish_word_list):
        #print(f"{key} 해당 키는 매파적입니다")
        Word_Emotion_dict["hawkish"].append(key)

print("학습 단어 극성 분류 사전 생성 하였습니다")
print("새로운 문서에 대한 스코어 계산을 시작합니다")

start_date = "2013-05-09"
def select_MPB(target_date):
    def date_range(start, end):
        start = datetime.strptime(start, "%Y-%m-%d")
        end = datetime.strptime(end, "%Y-%m")
        before_one_day = end - relativedelta(days=30)
        dates = [(start + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((before_one_day-start).days+1)]
        return dates
    dates = date_range(start_date, target_date)

    path = "./MPB_crawling_dataset"
    file_list = []
    if os.path.isdir(path):
        text_list = os.listdir(path)
        for i in text_list:
            if i[-3:] == "txt":
                target_file = i[:-4]
                if target_file in dates:
                    file_list.append(i)
    MPB_file = file_list[-1]                
    f = open(f"{path}/{MPB_file}", 'r')
    file_text = f.readline()
    return file_text

file_text = select_MPB(target_date)
target_text = (phrases[file_text.split()])
print("target_text 생성")

# 새로운 문서에 대한 스코어 계산
new_word_doveish_score = 0
new_word_hawkish_score = 0
for token in target_text:
    if token in Word_Emotion_dict['doveish']:
        new_word_doveish_score += 1 
    elif token in Word_Emotion_dict['hawkish']:
        new_word_hawkish_score += 1
print(new_word_doveish_score, new_word_hawkish_score)
if new_word_doveish_score >  new_word_hawkish_score:
    print(f"{target_date} 해당 날짜는 비둘기적입니다")
    #return f"{target_date} 해당 날짜는 비둘기적입니다"
elif new_word_doveish_score ==  new_word_hawkish_score:
    print("{target_date} 해당 날짜는 중립 성향 입니다")
    #return f"{target_date} 해당 날짜는 중립 성향 입니다"
elif new_word_doveish_score <  new_word_hawkish_score:
    print(f"{target_date} 해당 날짜는 매파적입니다")
    #return f"{target_date} 해당 날짜는 매파적입니다"


모델 생성 하였습니다
극성 기준 사전 생성 하였습니다
학습 단어 극성 분류 사전 생성 하였습니다
새로운 문서에 대한 스코어 계산을 시작합니다
target_text 생성
634 1253
2015-07 해당 날짜는 매파적입니다


In [ ]:
#최종 함수
def model_start(target_date,data_df): 
    start_date = "2013-05-09"
    def select_MPB(target_date):
        def date_range(start, end):
            start = datetime.strptime(start, "%Y-%m-%d")
            end = datetime.strptime(end, "%Y-%m")
            before_one_day = end - relativedelta(days=30)
            dates = [(start + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((before_one_day-start).days+1)]
            return dates
        dates = date_range(start_date, target_date)

        path = "./MPB_crawling_dataset"
        file_list = []
        if os.path.isdir(path):
            text_list = os.listdir(path)
            for i in text_list:
                if i[-3:] == "txt":
                    target_file = i[:-4]
                    if target_file in dates:
                        file_list.append(i)
        MPB_file = file_list[-1]                
        f = open(f"{path}/{MPB_file}", 'r')
        file_text = f.readline()
        return file_text

    file_text = select_MPB(target_date)
    target_text = (phrases[file_text.split()])
    print("target_text 생성")


    # 넘파이 코사인 유사도 계산
    def cos_sim(A, B):
        return dot(A, B)/(norm(A)*norm(B))

    def ngram2vec_train(data_df):
        X_train = data_df

        # 2. 특성 추출 (형태소 분석 및 N-그램)
        vocab = [token.split() for token in X_train]

        # 3. N-그램 추출
        phrases = Phrases([x for x in vocab], min_count=1, threshold=1.1)
        ngram_X_train = [' '.join(phrases[x]) for x in vocab]
        New_test_ngrams = ngram_X_train

        # 4. Ngram2Vec 모델 학습
        model = Word2Vec([x.split() for x in New_test_ngrams], vector_size=300, window=5, min_count=25)
        return model
    model = ngram2vec_train(data_df)
    print("모델 생성 하였습니다")

    # 극성 기준 사전
    doveish_word_list = []
    hawkish_word_list = []
    for key in seed_word_dict['doveish']:
        try:
            target = model.wv[key]
            doveish_word_list.append(key)
        except:
            pass

    for key in seed_word_dict['hawkish']:
        try:
            target = model.wv[key]
            hawkish_word_list.append(key)
        except:
            pass
    model_keys = model.wv.index_to_key
    print("극성 기준 사전 생성 하였습니다")

    # 카운팅할 극성 워드 생성
    Word_Emotion_dict = {"doveish":[],"hawkish":[]}
    for key in model_keys:
        doveish_score = 0
        hawkish_score = 0
        target = model.wv[key]
        for word in doveish_word_list:
            doveish_target = model.wv[word]
            doveish_score += cos_sim(target, doveish_target)
        for word in hawkish_word_list:
            hawkish_target = model.wv[word]
            hawkish_score += cos_sim(target, hawkish_target)
        if doveish_score / len(doveish_word_list) > hawkish_score / len(hawkish_word_list):
            #print(f"{key} 해당 키는 비둘기적입니다")
            Word_Emotion_dict["doveish"].append(key)
        elif doveish_score / len(doveish_word_list) == hawkish_score / len(hawkish_word_list):
            print(f"{key} 해당 키는 중립적입니다")
        elif doveish_score / len(doveish_word_list) < hawkish_score / len(hawkish_word_list):
            #print(f"{key} 해당 키는 매파적입니다")
            Word_Emotion_dict["hawkish"].append(key)

    print("학습 단어 극성 분류 사전 생성 하였습니다")
    print("새로운 문서에 대한 스코어 계산을 시작합니다")

    # 새로운 문서에 대한 스코어 계산
    new_word_doveish_score = 0
    new_word_hawkish_score = 0
    for token in target_text:
        if token in Word_Emotion_dict['doveish']:
            new_word_doveish_score += 1 
        elif token in Word_Emotion_dict['hawkish']:
            new_word_hawkish_score += 1
    print(new_word_doveish_score, new_word_hawkish_score)
    if new_word_doveish_score >  new_word_hawkish_score:
        return f"{target_date} 해당 날짜는 비둘기적입니다"
    elif new_word_doveish_score ==  new_word_hawkish_score:
        return f"{target_date} 해당 날짜는 중립 성향 입니다"
    elif new_word_doveish_score <  new_word_hawkish_score:
        return f"{target_date} 해당 날짜는 매파적입니다"


In [ ]:
#결과값
model_start(target_date,x_tain)

In [ ]:
#정확도 계산
# 콜금리기준으로 맞춘 스코어와 못맞춘 스코어를 계산합니다.

#1 극성 페이지 추출
def make_Emotion_dict():
    f = open(f"./call_rate_months.csv", 'r')
    file_lines = f.readlines()
    date = file_lines[0].split(",")
    rate = file_lines[1].split(",")
    date = date[:-4]
    rate = rate[:-4]
    date_dict = {date[idx]: rate[idx] for idx in range(len(rate))}
    target_date_dict = {}
    for i in date_dict:
        temp = i.replace("/",".")
        target_date_dict[temp] = date_dict[i]
    key_list = [key for key, value in target_date_dict.items()]
    value_list = [value for key, value in target_date_dict.items()]
    Emotion_dict = {"doveish":[], "hawkish":[]}
    for idx in range(len(key_list)-1):
        target_date = key_list[idx]
        target_rate = value_list[idx]
        compare_date = key_list[idx+1]
        compare_rate = value_list[idx+1]
        threshold_temp = float(target_rate) - float(compare_rate)
        threshold_temp = (threshold_temp**2)**0.5
        if threshold_temp > 0.001:
            if target_rate > compare_rate:
                Emotion_dict["doveish"] += [[target_date, compare_date]]
            else:
                Emotion_dict["hawkish"] += [[target_date, compare_date]]
    return Emotion_dict

Emotion_dict = make_Emotion_dict()
Emotion_dict

MPB_dict = {"doveish":[], "hawkish":[]}
for Emotions in Emotion_dict:
    if Emotions == "doveish":
        Emotion = list(Emotion_dict[Emotions])
        for date in Emotion:
            start = date[0]
            end = date[1]
            start_time = datetime.strptime(start, "%Y.%m")
            end_time_1 = datetime.strptime(end, "%Y.%m")
            end_time_2 = end_time_1 - timedelta(days=1)
            for day in range((end_time_2-start_time).days+1):
                target = (start_time + timedelta(days=day)).strftime("%Y-%m-%d")
                MPB_Path = f"./MPB_crawling_dataset/{target}.txt"
                file_name = f"{target}.txt"
                if os.path.isfile(MPB_Path):
                     MPB_dict["doveish"].append(file_name)
                     
    else:
        Emotion = list(Emotion_dict[Emotions])
        for date in Emotion:
            start = date[0]
            end = date[1]
            start_time = datetime.strptime(start, "%Y.%m")
            end_time_1 = datetime.strptime(end, "%Y.%m")
            end_time_2 = end_time_1 - timedelta(days=1)
            for day in range((end_time_2-start_time).days+1):
                target = (start_time + timedelta(days=day)).strftime("%Y-%m-%d")
                MPB_Path = f"./MPB_crawling_dataset/{target}.txt"
                file_name = f"{target}.txt"
                if os.path.isfile(MPB_Path):
                    MPB_dict["hawkish"].append(file_name)

total_len = 0
total = {"answer" : 0, "wrong answer" : 0}

for i in MPB_dict['doveish']:
    total_len += 1
    target_MPB_Path = f"./MPB_crawling_dataset/{i}"
    f = open(f"{target_MPB_Path}", 'r')
    accuracy_text = f.readline()
    accuracy_target = (phrases[accuracy_text.split()])
    new_word_doveish_score = 0
    new_word_hawkish_score = 0
    for token in accuracy_target:
        if token in Word_Emotion_dict['doveish']:
            new_word_doveish_score += 1 
        elif token in Word_Emotion_dict['hawkish']:
            new_word_hawkish_score += 1
    print(f"비둘기 점수: {new_word_doveish_score}, 매파 점수: {new_word_hawkish_score}")
    if new_word_doveish_score >  new_word_hawkish_score:
        total["answer"] += 1
    else:
        total["wrong answer"] += 1

for i in MPB_dict["hawkish"]:
    total_len += 1
    target_MPB_Path = f"./MPB_crawling_dataset/{i}"
    f = open(f"{target_MPB_Path}", 'r')
    accuracy_text = f.readline()
    accuracy_target = (phrases[accuracy_text.split()])
    new_word_doveish_score = 0
    new_word_hawkish_score = 0
    for token in accuracy_target:
        if token in Word_Emotion_dict['doveish']:
            new_word_doveish_score += 1 
        elif token in Word_Emotion_dict['hawkish']:
            new_word_hawkish_score += 1
    print(f"비둘기 점수: {new_word_doveish_score}, 매파 점수: {new_word_hawkish_score}")
    if new_word_doveish_score <  new_word_hawkish_score:
        total["answer"] += 1
    else:
        total["wrong answer"] += 1
    if new_word_doveish_score >  new_word_hawkish_score:
        total["answer"] += 1

print(f"정답률 : {total['answer'] / total_len}, 오답률 :{total['wrong answer'] / total_len}")
                    

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
sentence_tone = (hawkish_count - dovish_count) / (hawkish_count + dovish_count)

In [ ]:
tone_and_policy_rate_df.corr(method='pearson')